In [1]:
from sys import path
path.append('..//modules')
import psycopg2 as pg
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import create_engine, MetaData, Table
import pandas.io.sql as psql
import pandas as pd
import time
#create a postgres engine and a pg connection
from modules.db_connector import *
print("DB connecting engine imported")

DB connecting engine imported


In [1]:
l_dct = [
{"layer_name": "chranene_objekty:chranene_uzemia_mchu", "service": "https://www.sopsr.sk/geoserver/chranene_objekty/ows?", "db_table": "sk_mchu_sps_sopsr"},
{"layer_name": "chranene_objekty:chranene_uzemia_vchu", "service": "https://www.sopsr.sk/geoserver/chranene_objekty/ows?", "db_table": "sk_vchu_lps_sopsr"}, 
{"layer_name": "sz:chranene_uzemia_np_s_op", "service": "https://www.sopsr.sk/geoserver/sz/ows?", "db_table": "sk_np_w_pz_sopsr"},
{"layer_name": "chranene_objekty:chranene_uzemia_biosfericke_rezervacie", "service": "https://www.sopsr.sk/geoserver/chranene_objekty/ows?", "db_table": "sk_br_sopsr"},
{"layer_name": "chranene_objekty:chranene_uzemia_uev", "service": "https://www.sopsr.sk/geoserver/chranene_objekty/ows?", "db_table": "sk_uev_teui_sopsr"},
{"layer_name": "chranene_objekty:chranene_uzemia_chvu", "service": "https://www.sopsr.sk/geoserver/chranene_objekty/ows?", "db_table": "sk_chvu_bpa_sopsr"}
]


In [18]:
#import Sal(2023) module for calling WFS to gpdf
from modules import call_wfs
print("Module call_wfs imported")

start = time.time()
for dct in l_dct:
    print("Making call to WFS: ", dct["layer_name"])
    #extract layer from WFS and transform to a gpdf containing all SK municipalities (it takes about 2 minutes)
    gpdf_gku = call_wfs.wfs2gp_df(dct["layer_name"], dct["service"])
    print("gpdf_gku "+ dct["layer_name"] +" retrieved in : ", round(time.time() - start, 2), " seconds.")

    #load the created gpdf in to postgis db
    gpdf_gku.to_postgis(dct["db_table"], engine, schema = 'gtlab', if_exists='replace', index=False, 
                   dtype={'geom': Geometry(geometry_type='MultiPolygon', srid= '5514')})
    print(dct["layer_name"] + " loaded in db under name " + dct["db_table"] + " in: ", round(time.time() - start, 2), " seconds.")

Module call_wfs imported
Making call to WFS:  chranene_objekty:chranene_uzemia_mchu
1160
gpdf_gku chranene_objekty:chranene_uzemia_mchu retrieved in :  25.75  seconds.
chranene_objekty:chranene_uzemia_mchu loaded in db under name sk_mchu_sps_sopsr in:  26.38  seconds.


In [30]:
l_alter_stmts = ['ALTER TABLE gtlab.sk_mchu_sps_sopsr RENAME COLUMN "geometry" to geom;', 
'ALTER TABLE gtlab.sk_vchu_lps_sopsr RENAME COLUMN "geometry" to geom;',
'ALTER TABLE gtlab.sk_np_w_pz_sopsr RENAME COLUMN "geometry" to geom;',
'ALTER TABLE gtlab.sk_br_sopsr RENAME COLUMN "geometry" to geom;',
'ALTER TABLE gtlab.sk_uev_teui_sopsr RENAME COLUMN "geometry" to geom;',
'ALTER TABLE gtlab.sk_chvu_bpa_sopsr RENAME COLUMN "geometry" to geom;']

for stmt in l_alter_stmts:
    with engine.connect() as con1:
        con1.execute(text(stmt))
        con1.commit()
        con1.close()



l_updt_stmts = ["SELECT UpdateGeometrySRID('gtlab','sk_mchu_sps_sopsr', 'geom', 5514);",
"SELECT UpdateGeometrySRID('gtlab','sk_vchu_lps_sopsr', 'geom', 5514);",
"SELECT UpdateGeometrySRID('gtlab','sk_np_w_pz_sopsr', 'geom', 5514);",
"SELECT UpdateGeometrySRID('gtlab','sk_br_sopsr', 'geom', 5514);",
"SELECT UpdateGeometrySRID('gtlab','sk_uev_teui_sopsr', 'geom', 5514);",
"SELECT UpdateGeometrySRID('gtlab','sk_chvu_bpa_sopsr', 'geom', 5514);"]

for stmt in l_updt_stmts:
    with engine.connect() as con1:
        con1.execute(text(stmt))
        con1.commit()
        con1.close()

In [3]:
list_alter_stmts = ["ALTER TABLE gtlab.sk_sgs_vgl_sguds ADD COLUMN nmskbnm_prob numeric;"]
                    #gml_id_cat7_sk_mchu_sps_sopsr text"]
                    
                    #ADD COLUMN category_name_sk_mchu_sps_sopsr_by_name text, ADD COLUMN siteid_sk_mchu_sps_sopsr_by_name text, add column name_sk_mchu_sps_sopsr_by_name text;"]
                    
                    #ADD COLUMN siteid_cat8_sk_vchu_lps_sopsr text, ADD COLUMN siteid_cat9_sk_vchu_lps_sopsr text, ADD COLUMN siteid_cat10_sk_vchu_lps_sopsr text;"]
                    
                    
                    #ADD COLUMN gml_id_cat1_sk_mchu_sps_sopsr text, ADD COLUMN gml_id_cat2_sk_mchu_sps_sopsr text, ADD COLUMN gml_id_cat3_sk_mchu_sps_sopsr text, ADD COLUMN gml_id_cat4_sk_mchu_sps_sopsr text, ADD COLUMN gml_id_cat5_sk_mchu_sps_sopsr text, ADD COLUMN gml_id_cat6_sk_mchu_sps_sopsr text ;"] 
                    
                    #add column gml_id_sk_np_w_pz_sopsr text, add column siteid_sk_uev_teui_sopsr text, add column siteid_sk_chvu_bpa_sopsr text;"]

for stmt in list_alter_stmts:
    with engine.connect() as con1:
        con1.execute(text(stmt))
        con1.commit()
        con1.close()

In [3]:
with engine.connect() as con1:
    q1 = "select * from gtlab.sk_sgs_vgl_sguds order by id"
    q2 = """
        select gml_id, "SITETITLE_SK", "SITETITLE_EN" nm_en, "CATEGORY", geom from gtlab.sk_mchu_sps_sopsr where "CATEGORY" not in (6,7)
    
    """
    q3 = 'select gml_id, "SITETITLE_SK", "SITETITLE_EN" nm_en, "CATEGORY", geom from gtlab.sk_vchu_lps_sopsr'
    df_vgl = pd.read_sql(q1, con=con1)
    df_mchu = pd.read_sql(q2, con=con1)
    df_vchu = pd.read_sql(q3, con=con1)
    con1.close()
    


In [2]:
from modules import get_sopsr_ids
#get_sopsr_ids.get_sopsr_id_mchu_cat1(df_vgl)
#get_sopsr_ids.get_sopsr_id_mchu_cat2(df_vgl)
#get_sopsr_ids.get_sopsr_id_mchu_cat3(df_vgl)
#get_sopsr_ids.get_sopsr_id_mchu_cat4(df_vgl)
#get_sopsr_ids.get_sopsr_id_mchu_cat5(df_vgl)
#get_sopsr_ids.get_sopsr_id_mchu_cat6(df_vgl)
#get_sopsr_ids.get_sopsr_id_mchu_cat7(df_vgl)

#get_sopsr_ids.get_sopsr_id_vchu_cat8(df_vgl)
#get_sopsr_ids.get_sopsr_id_vchu_cat9(df_vgl)
#get_sopsr_ids.get_sopsr_id_vchu_cat10(df_vgl)

#get_sopsr_ids.get_sopsr_id_np_w_pz(df_vgl)
#get_sopsr_ids.get_sopsr_id_uev_teui(df_vgl)
#get_sopsr_ids.get_sopsr_id_chvu_bpa(df_vgl)

#get_sopsr_ids.get_sospr_mchu_by_name_penetration(df_vgl, df_mchu)
get_sopsr_ids.measure_dist_vgl_mchu_by_name()


